# Lesson 1 - Calibration Overview

# Dynamically Dimensioned Search:
Dynamically Dimensioned Search (DDS) algorithm is used as the optimization algorithm for IWAAs where the search strategy in model parameter space is scaled to the maximum number of iterations specified by the user. In other words, in initial iterations the algorithm searches globally and as the procedure approaches the maximum user-defined number of iterations, the search transitions from a global to a local search. This transition from a global to local search is achieved by dynamically and probabilistically reducing the search dimension, which is the subset of the calibration parameters that will be updated in a given iteration. The DDS algorithm works as follows:

**Step 1**: Define the DDS inputs:
* Maximum number of iterations (`m`). The maximum number of iterations is set to 400 except for the domains that were too large (> 5000 km2) in that case 200 iterations were used for the calibration.  
* Initial solution x0 = [x1, ...., xD] where `D` is the number of calibration parameters. Calibration parameters and default values used will be discussed in detail in this lesson. 
* Vector of lower (xmin) and upper bounds (xmax) for all the calibration parameters. Range of the calibration parameters will be discussed in detail in this lesson. 
* Neighborhood perturbation size parameter (r) that defines the random perturbation standard deviation as a function of calibration parameter range. Tolson and Shoemaker 2007 suggested to use a value of 0.2 as default, and reducing this parameter is only recommended if the calibration procedure is starting from a good initial solution or a calibration exercise is resumed to refine the solution even further. Since in each version upgrade, we start the calibration with the default parameters, we used the suggested r = 0.2 for all the versions. 

**Step 2**: In iteration 1, run WRF-Hydro for the calibration basin using the default parameters and calculate the objective function and set those as a best parameter set (`xbest`) and best objective function (`Fbest`). 

**Step 3**: Randomly select `J` of the `D` calibration parameters to be included in the search space. The probability of a parameter to be chosen for inclusion in search is equal to `P(i) = 1 - ln(i)/ln(m)`, therefore the possibility of a parameter to be chosen reduces with increase in iteration numbers. Figure below shows a sample of search dimension (number of parameters selected and updated) evolution with iteration for an exercise where 15 parameters are calibrated. 

**NOTE:** if the DDS search dimension is less than 1, it will be set to 1 and one parameter will be perturbed. 

<p style="text-align:center;">
<img src="./images/DDS_search.png" width="600" height="600" />
</p>

**Step 4**: For j = 1, ...., J perturb the `Xbestj`.If the new perturbed parameter is outside the parameter range, it will be adjusted to fall into the range. 

**Step 5**: Run NWM with the new parameter set, and update if the objective function has improved upon Fbest.

**Step 6**: Update iteration count, if iteration count is less than m then go to Step 2, otherwise exit the calibration procedure. 


# Programs & Workflow:
Calibration workflow runs a series of six individual python scripts, typically run from the command line (terminal). The python scripts must be run in order, and only once the previous script has completed. Brief descriptions of each script are provided below and a more detailed explanation will be provided in the upcoming lessons. 

<p style="text-align:center;">
<img src="./images/Calibration_workflow_2021_image.png" width="600" height="600" />
</p>

**initDB.py:**

This program is run once to initialize and set up the calibration database and associated tables used during the experiment. Completing this step will create an empty database with all the relevant tables that will be filled with information provided during the calibration procedure. 


**inputDomainMeta.py:**

This program reads in a CSV file you will need to fill out that describes modeling domains to be used for calibration. This information is entered into the database for later workflow use. More description on the CSV will occur during the setup section.

**jobInit.py:** 

This program is run to establish a calibration ‘experiment’. The program reads a config file 
(explained in depth below) and sets up the necessary run directories, paths to necessary files, and inputs associated metadata into the database. Upon successful completion, the program will return a message that the workflow for the unique job ID value is been setup. The job ID is defined by the user and will be used in subsequent programs to run the calibration.

**spinOrchestrator.py**

 This is the first program that is run to initialize the calibration experiment. The main purpose of this program is to run the NWM/WRF-Hydro spinup for all domains being calibrated. This program needs to successfully complete before moving onto the next step. 


**calibOrchestrator.py:** 

This is the second program that is run in the calibration workflow. This program runs the main workflow to adjust parameter values, execute interim model simulations, evaluate model output against observations, and further adjust parameter values. This program must be completed successfully before moving onto the next step.

**runValidOrchestrator.py:**

This is the third and final main program in the calibration workflow. This program manages running the model with the final calibrated parameters over a specified evaluation period for the evaluation of the parameters.


# Pre Calibration Step 1: Configure Setup File: "setup.parm"
The primary file you will be editing in preparation for setting up a calibration workflow job is the `setup.parm` file. It is best to think of this file as a master configuration file to guide the workflow. This file contains multiple options that define how the workflow will submit jobs for models/analysis, which basins to calibrate from the database, methods for reporting errors to the user, model physics options, and paths to general parameter files and executables.
The `setup.parm` file is divided up into sections: 
* *logistics*: contains all the options related to calibration options and calibration job submission. 
* sensitivity: contains all the options related to sensibility analysis. 
* gageInfo: user could specify which gages to calibrate.  
* lsmPhysics: contains all NoahMP physics options.  
* forcing: contains the forcing type. 
* modelTime: contains all the option related to the frequency of outputting.
* hydroIO: contains all the options related to hydro outputting. 
* hydroPhysics. contains all the hydro physics options. 

Let's take a look at the this file and its content: 

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/PyWrfHydroCalib/setup_files/setup.parm

# Description of the setup file content. 
As mentioned above the setup file had different sections which each target a specific part of the model calibration. Below we are providing a short description of each section and its arguments. 

## logistics section: 
The first section of the setup.parm file is ‘logistics’, which guides the workflow. This section has all the information about the machine specific options, what components of the model to calibrate(streamflow/snow/soil moisture), how to define the objective functions, what the dates are for each spinup/calibration/validation and more. 

|Argument|Required|Description|
|-|-|-|
| **outDir** | Req. |Where do you want your calibration experiment to be constructed? |
|**expName** | Req. |What the name of your experiment is. This can be anything you want. |
|**acctKey** | Opt. |If you are running on a que system that requires credentials to submit a job, specify your account key here. |
|**optQueName** | Opt. |If you need to direct model simulations/jobs to a specific que, you can specify that here. |
|**nCoresModel** | Req. |How many CPUs are you running your model simulations over? |
|**nNodesModel** | Req.| If you are running across multiple nodes on a que system, specify here.| 
|**nCoresPerNode** | Req. |If you are running on a que system, specify how many CPU cores you have available per node. |
|**runSens** | Req.| Are we running sensitivity analysis? 0 - No, 1 - Yes |
|**sensParmTbl** | Req.| Path to the table of parameter values to use in sensitivity analysis. Same format as the calibration parameter table. |
|**runCalib** | Req. |Are we running calibration? 0 - No, 1 - Yes |
|**calibParmTbl** | Req.| Path to the table of parameter values to use in calibration. |
|**moduleLoadStr**| Req. | List of the modules|
|**dailyStats** | Req. | Flag to direct worfklow to calculate error statistics on a daily scale, instead of hourly by default. Specify 1 to activate. |
|**dbBackup** |Req. | Flag to turn on/off database backup. If on, the databse will be locked and backed up once an hour during execution to the job directory output file.|
|**coldStart** | Req. |Flag to direct calibration workflow to cold start your model simulation for each iteration during calibration. Specify 1 to activate. |
|**optSpinFlag** | Req. | Flag to direct the workflow to use an alternative spinup file already in place in the input directory for the basins being calibrated. This allows the user to bypass the spinup step. Specify 1 to activate. |
|**stripCalibOutputs** | Req. |If you desire to omit outputs during an initial window for each model iteration, you can specify 1 here to activate this feature. This was designed to minimize I/O model burdens. |
|**stripCalibHours** | Req. |Specify an initial window in hours to strip outputs. This is only used if stripCalibOutputs has been activated. |
|**jobRunType** | Req. |This will specify how the model simulations and calibration code being executed. |
|**mpiCmd** | Req. |What is the MPI command being used to execute the model simulations. This is required, as the MPI command is also used in job scheduler scripts. |
|**cpuPinCmd** |Opt. |If you are running on a job scheduler, how do you want to pin specific model simulations on a compute node? This put in to allow multiple basins on one node. |
|**numIter** | Req. |How many model iterations would you like to run your calibration experiment over? |
|**calibMethod** | Req.| Right now only DDS is allowed. Future upgrades will incorporate additional calibration methods.|
|**enableStreamflowCalib**| Req.| Specify whether we are calibrating to streamflow, 0- not calibrating, 1 - calibrating | 
|**enableSnowCalib**| Req. | Specify if snow input data will be used in calibration, 0- not calibrating, 1 -calibrating.|
|**enableSnowCalib**| Req. | Specify if soil moisture input data will be used in calibration, 0- not calibrating, 1 -calibrating.|
|**streamflowObjectiveFunction** | Req.|Select what error metric for streamflow to minimize during the calibraion experiment. |
|**snowObjectiveFunction** | Req.|Select what error metric for snow to minimize during the calibraion experiment. |
|**soilMoistureObjectiveFunction** |Req.|Select what error metric for soil moisture to minimize during the calibraion experiment. |
|**streamflowWeight**| Req. | Multiplier to streamflow objective function|
|**snowWeight**| Req.|Multiplier to the snow objective function|
|**soilMoistureWeight** |Req.|Multiplier to soil moisture objective function|
|**basinType**| Req. | Type of the basin is used in event identification, 0 - snowy, 1- slow, 2-flashy, 3- regular|
|**weight1Event**| Req. | weight for peak bias used in event based metric (combined peak bias and volume bias)|
|**weight2Event**|Req.  | weight for volume bias used in event based metric (combined peak bias and volume bias)|
|**ddsR** | Req.| This is a DDS-specific parameter that tunes how random values are generated for each iteration. |
|**enableMask**|Req.| Specify 1 to use mask covering part of basin and not calibrating that part, or 0 otherwise (calibrating all basin)|
|**enableMultiSites**|Req.| Specify 1 to calibrate to more than 1 streamgage simultaneously|
|**email** | Req. |Where do you want status and error messages to be directed to?|
|**wrfExe** | Req.| Path to the WRF-Hydro executable to be used in the workflow. ]
|**genParmTbl** | Req. |Path to the GENPARM.TBL file used by the model. |
|**mpParmTbl** | Req. |Path to the MPARM.TBL file used by the model. |
|**urbParmTbl** |Opt. |Path to the URBPARM.TBL file used by the model. |
|**vegParmTbl** | Opt. |Path to the VEGPARM.TBL file used by the model. |
|**soilParmTbl** | Req. |Path to the SOILPARM.TBL file used by the model. |
|**bSpinDate** | Opt. |Beginning date for the spinup. |
|**eSpinDate** | Opt. |Ending date for the spinup. |
|**bCalibDate** | Req. |Beginning date for each calibration iteration. |
|**eCalibDate** | Req. |Ending date for each calibration iteration. |
|**bCalibEvalDate** | Req.| The date within each calibration iteration to begin analysis. |
|**bValidDate** | Opt. |Beginning date for the validation simulation. |
|**eValidDate** | Opt. |Ending date for the validation simulation. |
|**bValidEvalDate** | Opt. |The date within the validation simulation to begin analysis. |


## Remaining Sections: 
**Sensitivity**: Contains all the options relevant to the sensitivity analysis. 

**gageInfo**: User could specify a subset of gages to be calibrated. 

**lsmPhysics**: LSM physics options as well as other LSM related options. 

**forcing**:User defined forcing type. 

**modelTime**: contains all the options that control model temporal resolution (LSM), the frequency of model outputting and restart files frequency. 

**hydroIO**: Contains all the options related to the hydro outputting.

**hydroPhysics** : Hydro physics options as well as few other related options such as model time step. 

## Pre Calibration Step 2: Configure Calibration Parameter Selection "calib_parms.tbl"
In addition to the `setup.parm` file, the `calib_parms.tbl` file is needed to direct the workflow to determine which model parameters will be calibrated, along with the range of parameter values to be used and the default value for the parameter. A template table is located under `PyWrfHydroCalib/setup_files/calib_parms.tbl` which you can copy and edit for your own calibration experiment.


In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/PyWrfHydroCalib/setup_files
cat calib_params.tbl

Within this table, you will find:

- Parameter -- all the potential parameters to be calibrated
- calib_flag -- 1 or 0. This flag will turn calibration on (1) for that parameter or off (0). 
- minValues -- The minimum range value for parameter calibration
- maxValues -- The maximum range value for parameter calibration
- ini -- specifies the default values to be used for either default un-calibrated values, or the initial values going into the calibration workflow. 

It should be noted that it is up to user to determine what range of the parameter value is best for your calibration experiment, the suggested range above is based on the previous literature review and our team's best estimate. 

We have summarized the parameters that were used in IWAA calibration in the below table. 

|Name|Units|Multiplier|Description|
|-|-|-|-|
|**BEXP** | dimensionless| Yes | *Pore size distribution index*: BEXP controls the shape of the soil water retention curve, and therefore how slowly/quickly water will move through the soil column. It can have any positive values where small values indicate the medium having a wide range of pore sizes and high values indicating more uniform pore size distribution. Higher values of the BEXP will result in higher effective saturation values and consequently higher relative hydraulic conductivity. With higher BEXP values, the water will move faster and one would observe higher peak values in the hydrographs and lower baseflow values at interevents. |
|**SMCMAX** | volumetric fraction| Yes | *Saturation soil moisture content (i.e., porosity)*|
|**DKSAT** |m/s | Yes | *Saturated hydraulic conductivity*:As with most physically-based hydrological models, the soil saturated hydraulic conductivity (DKSAT) controls the speed at which water moves through the subsurface. This is a sensitive parameter in the model, and while easy to measure at the point scale, DKSAT is tricky to estimate at the scale of kilometers. Initial values are estimated based on soil texture class, but reported ranges have large (many orders of magnitude) variability.|
|**RSURFEXP** | dimensionless| No | *Exponent in the resistance equation for soil evaporation*: RSURFEXP controls the shape for the resistance curve as it relates to soil moisture, higher RSURFEXP will result in larger resistance for a given soil moisture and hence less soil evaporation.|
|**SLOPE** |unitless | No |  *Linear scaling of "openness" of bottom drainage boundary*: This is another important parameter that affects water partitioning in Noah/NoahMP. Originally estimated based on land surface topography (hence the name SLOPE), the SLOPE parameter actually controls how open or closed the bottom boundary of the soil column is. Values range from 0 to 1, where 0 is a completely closed bottom boundary and 1 is completely open. Lower SLOPE values will keep more water in the soil column, while higher values will allow more water to drain to the channel or to deeper baseflow stores, depending on the selected baseflow physics options.|
|**RETDEPRTFAC** | unitless | No |  *Multiplier on retention depth limit*: Ponded water on the surface above this retention depth threshold can be moved around the landscape via overland flow. Maximum retention depth is variable by terrain slope where higher slopes have lower maximum retention depth, while flat areas have higher initial values. The default value in the NWM code is quite small (~0.001mm), however, in many regions landscape features like wetlands, small detention ponds, and heavy vegetation litter/debris can trap water on the land surface. Increasing the RETDEPRTFAC multiplier will hold more ponded water on the surface before it becomes runoff.|
|**LKSATFAC** |unitless | No |  *Multiplier on lateral hydraulic conductivity (controls anisotropy between vertical and lateral conductivity)*: By default, lateral conductivity matches vertical conductivity. However, in the real world we frequently see many orders of magnitude higher conductivities in the lateral direction vs. the vertical direction (due to soil stratigraphy, preferential flow paths caused by roots and animals, etc.). LKSATFAC is a parameter to adjust the lateral saturated hydraulic conductivity and account for this anisotropy.|
|**ZMAX** | mm | No | *Maximum groundwater bucket depth before "spilling" occurs.* It should be noted that ZMAX has no physical basis as the bucket model is a simple conceptualized model |
|**EXPON** |dimensionless | No | *Exponent controlling rate of bucket drainage as a function of depth*|
|**CWPVT** | 1/m | Yes | *Canopy wind parameter for canopy wind profile formulation*: It is a canopy wind parameter absorption/attenuation which helps to remove saturated air from the canopy. Higher value meaning more wind attenuation from the canopy, which reduces the transpiration. Higher CWPVT values also reduce snow sublimation which result in more snow in the pack and increasing SWE as well as pushing the melt season slightly further back in the season. |
|**VCMX25** | unol/m2/s | Yes |  Maximum carboxylation at 25C|
|**MP** | unitless | Yes |  *Slope of Ball-Berry conductance relationship*: larger values of mp indicates the leaf consumes more water to produce the same carbon mass and therefore will have greater transpiration|
|**MFSNO** | dimensionless | Yes | *Melt factor for snow depletion curve* (relationship between the snow covered fraction and snow depth) in the melting season. Increasing MFSNO yields a smaller snow cover fraction for the same snow height, reduces SWE in the snowpack and moves the timing of snowmelt forward in time. Therefore, the streamflow peaks are earlier in time and lower for larger values of MFSNO.|
|**AXAJ**|unitless | Yes|Tension water distribution inflection parameter |
|**BXAJ**| unitless| Yes|Tension water distributeion shape parameter |
|**XXAJ**| unitless| Yes| Free water distribution shape parameter |
|**NEXP**|unitless | No|Exponent in the decay function for lateral Ksat over depth |

### Absolute value versus multipliers: 
Some of these parameters are a function of vegetation type (`CWPVT, VCMX25, MP`) or soil type (`BEXP, SMCMAX, DKSAT`), and therefore we use a multiplier number in calibration to preserve the spatial pattern of those parameters. MFSNO is also formulated as a multiplier in the calibration code, in NWMv20 we had different starting value for short/tall vegetation. The rest of the parameters are substitute values and will overwrite the default values. 


## Conclusion:

Once the setup.parm file and calibration parameters are set and verified, we can begin the process of initializing databases for calibration. Proceed to *Lesson 2 - Create Database*